# 📧 Spam Classification with BERT

This notebook demonstrates a complete **transfer learning pipeline** using BERT for text classification. We'll fine-tune a pre-trained BERT model to classify SMS messages as spam or ham (not spam).

## Pipeline Overview

```text
┌─────────────────────────────────────────────────────────────────┐
│                    TRANSFER LEARNING PIPELINE                  │
│                                                                │
│  1. Load Data → 2. Preprocess → 3. Tokenize → 4. Create Model  │
│                                                                │
│  5. Train (freeze BERT) → 6. Evaluate → 7. Predict             │
└─────────────────────────────────────────────────────────────────┘
```

## Key Concepts

| Concept | Description |
|:--------|:------------|
| **Transfer Learning** | Use pre-trained BERT, only train classifier head |
| **Feature Extraction** | Freeze BERT weights, extract embeddings |
| **[CLS] Token** | Use the first token's embedding for classification |
| **Binary Classification** | Spam (1) vs Ham (0) |

---

## 1. Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd

### Check GPU Availability

BERT is computationally intensive, so we'll use GPU if available for faster training.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

---

## 2. Load and Explore the Dataset

We're using the **SMS Spam Collection** dataset, which contains labeled SMS messages.

In [ ]:
df = pd.read_csv('spam.csv')
df.head()

### Label Encoding

Convert text labels to numeric values:
- `ham` → `0` (legitimate message)
- `spam` → `1` (spam message)

In [ ]:
df.Category = df.Category.map({'ham':0, 'spam':1})
df.head()

### Class Distribution

Let's check the balance of our dataset:

In [ ]:
df.Category.value_counts()

**⚠️ Class Imbalance Detected!**

```text
Ham:  4825 (87%)
Spam:  747 (13%)
```

The dataset is heavily imbalanced. To address this, we'll:
1. Keep all spam messages (747)
2. Randomly sample 1000 ham messages

This creates a more balanced dataset for training.

In [ ]:
df_spam = df[df.Category == 1]
df_spam.shape

In [ ]:
df_ham_small = df[df.Category == 0].sample(n=1000, random_state=42)
df_ham_small.shape

In [ ]:
df_small = pd.concat([df_spam, df_ham_small])
df_small.Category.value_counts()

---

## 3. Train/Test Split

Split the data into training (80%) and validation (20%) sets.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_small.Message,
    df_small.Category,
    test_size=0.2,
    random_state=42
)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

---

## 4. Tokenization

We need to convert text into the format BERT expects:

```text
Input Text: "Click here to win!"
         ↓
    Tokenizer
         ↓
┌─────────────────────────────────────────────┐
│ input_ids:      [101, 11562, 2182, ...]    │ ← Token IDs
│ attention_mask: [1, 1, 1, 1, 0, 0, ...]    │ ← 1=real, 0=padding
└─────────────────────────────────────────────┘
```

**Key Parameters:**
- `max_length=128`: Truncate/pad all sequences to 128 tokens
- `padding='max_length'`: Pad shorter sequences
- `truncation=True`: Truncate longer sequences

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(texts, labels):
    encodings = tokenizer(texts, padding='max_length', max_length=128, truncation=True, return_tensors='pt')
    return encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels, dtype=torch.float)

# Example
tokenize_function(['Click here, Hurry', 'Let\'s have lunch tomorrow'], [1, 0])

### Tokenize All Data

In [ ]:
train_input_ids, train_attention_mask, train_labels = tokenize_function(X_train.values.tolist(), y_train.values.tolist())
val_input_ids, val_attention_mask, val_labels = tokenize_function(X_test.values.tolist(), y_test.values.tolist())

---

## 5. Create PyTorch DataLoaders

DataLoaders handle batching, shuffling, and efficient data loading.

In [ ]:
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = torch.utils.data.TensorDataset(val_input_ids, val_attention_mask, val_labels)

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

---

## 6. Build the Classification Model

Our model architecture:

```text
┌────────────────────────────────────────────────────────────┐
│                    SPAM CLASSIFIER                        │
│                                                           │
│   Input: [CLS] token₁ token₂ ... tokenₙ [SEP] [PAD] ...   │
│              ↓                                            │
│   ┌──────────────────────────────────────────────────┐     │
│   │           BERT (FROZEN)                         │     │
│   │   12 Transformer Encoder Layers                 │     │
│   │   Pre-trained weights (110M parameters)         │     │
│   └──────────────────────────────────────────────────┘     │
│              ↓ (extract [CLS] token embedding)            │
│   ┌──────────────────────────────────────────────────┐     │
│   │        CLASSIFIER HEAD (TRAINABLE)              │     │
│   │   Linear(768 → 256) → ReLU → Dropout(0.3)       │     │
│   │   Linear(256 → 1) → Sigmoid                     │     │
│   └──────────────────────────────────────────────────┘     │
│              ↓                                            │
│   Output: Spam Probability (0.0 to 1.0)                   │
└────────────────────────────────────────────────────────────┘
```

**Why freeze BERT?**
- BERT is already trained on massive data
- Training only the classifier head is faster
- Prevents overfitting on small datasets
- Reduces memory requirements

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        hidden_size = self.bert.config.hidden_size

        # Freeze BERT parameters (transfer learning)
        for param in self.bert.parameters():
            param.requires_grad = False

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask):
        # Get BERT output
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Extract [CLS] token embedding (position 0)
        sentence_embedding = bert_output.last_hidden_state[:,0,:]
        
        # Pass through classifier
        return self.classifier(sentence_embedding)

### Initialize Model, Optimizer, and Loss

In [ ]:
model = SentimentClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification

model = model.to(device)
criterion = criterion.to(device)

---

## 7. Training Loop

We'll train for 2 epochs (since BERT features are already good, we don't need many).

**Training Process:**
1. Forward pass through model
2. Calculate loss (BCE)
3. Backward pass (compute gradients)
4. Update classifier weights (BERT stays frozen)

In [ ]:
epochs = 2
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch, (input_ids, attention_mask, labels) in enumerate(train_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask).squeeze(dim=-1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f"Batch: {batch}, Epoch: {epoch}, Loss:  {loss.item():0.2f}")
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss}')

---

## 8. Evaluation

Evaluate the model on the validation set to measure accuracy.

In [ ]:
model.eval()
total_val_loss = 0
correct_predictions = 0

with torch.no_grad():
    for input_ids, attention_mask, labels in val_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device, dtype=torch.float)

        outputs = model(input_ids, attention_mask).squeeze(dim=-1)
        loss = criterion(outputs, labels.view_as(outputs))
        total_val_loss += loss.item()

        preds = (outputs > 0.5).float()
        correct_predictions += torch.sum(preds == labels)

avg_val_loss = total_val_loss / len(val_loader)
val_accuracy = correct_predictions.double() / len(val_dataset)
print(f'Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy:.4f}')

---

## 9. Inference on New Messages

Let's create a function to classify new messages:

In [ ]:
def predict_message(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask).squeeze(dim=-1)
        prob = output.item()
        label = 'spam' if prob > 0.5 else 'ham'
        return label, prob

### Test with Sample Messages

In [ ]:
# Test with a spam message
sample_text = "WINNER! You've won a luxury holiday! Call now to claim."
label, confidence = predict_message(sample_text, model, tokenizer, device)
print(f"Text: {sample_text}\nPrediction: {label} (Confidence: {confidence:.4f})")

In [ ]:
# Test with a ham message
sample_text = "Hey, are we still meeting for lunch tomorrow?"
label, confidence = predict_message(sample_text, model, tokenizer, device)
print(f"Text: {sample_text}\nPrediction: {label} (Confidence: {confidence:.4f})")

---

## 📚 Key Takeaways

| Concept | Description |
|:--------|:------------|
| **Transfer Learning** | Use pre-trained BERT as feature extractor |
| **Frozen Weights** | Don't update BERT parameters during training |
| **[CLS] Token** | First token captures sentence-level meaning |
| **Classifier Head** | Small trainable network on top of BERT |
| **BCE Loss** | Binary Cross Entropy for binary classification |
| **Fast Training** | Only 2 epochs needed with pre-trained features |

---

## 🚀 Possible Improvements

1. **Fine-tune BERT**: Unfreeze some layers for better performance
2. **Use BertForSequenceClassification**: Hugging Face's pre-built classifier
3. **Add more data**: Use the full dataset with class weighting
4. **Try different models**: DistilBERT for faster inference, RoBERTa for better performance